In [1]:
# Import Statemnts
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
import joblib


In [2]:
# Change these values to change the time period
Month = 1 
Hour = 12

# Read in the data
df = pd.read_csv("generator_level_lbmp.csv")

# Clean the data and create dummy variables
df = pd.get_dummies(df, prefix="Zone", columns=['Name']) # create dummy variables for zones
df.loc[df["power_source"] == "Uranium", "renewable"] = 0 # Uranium is not renewable 
if Month <= 10 and Month >= 5: # calculate max capacity based on season
    df["max_cap"] = df["summer_capacity"] * 1000 # GWh to MWH
else: 
    df["max_cap"] = df["winter_capacity"] * 1000 # GWh to MWH

df.head()

,Unnamed: 0,Month,Hour,Average LBMP,generator_name,summer_capacity,winter_capacity,power_source,Zone,renewable,...,Zone_DUNWOD,Zone_GENESE,Zone_HUD VL,Zone_LONGIL,Zone_MHK VL,Zone_MILLWD,Zone_N.Y.C.,Zone_NORTH,Zone_WEST,max_cap
0,0,1,1,32.722258,Allens Falls,4.4,4.4,Water,F,1,...,False,False,False,False,False,False,False,False,False,4400.0
1,1,1,1,32.722258,"Athens 1, 2, and 3",994.1,1202.8,Natural Gas,F,0,...,False,False,False,False,False,False,False,False,False,1202800.0
2,2,1,1,32.722258,Baldwinsville 1 and 2,0.6,0.6,Water,F,1,...,False,False,False,False,False,False,False,False,False,600.0
3,3,1,1,32.722258,Beacon LESR,0.0,0.0,Flywheel,F,0,...,False,False,False,False,False,False,False,False,False,0.0
4,4,1,1,32.722258,Beardslee 1 and 2,20.0,20.0,Water,F,1,...,False,False,False,False,False,False,False,False,False,20000.0


In [3]:
# Predict Demand for Day Ahead and Hour Ahead 
# Use the following features:
#   - Hour of Day
#   - Month of Year
#   - Zone


ALL_ZONES = ["Zone_CAPITL", "Zone_CENTRL", "Zone_DUNWOD", "Zone_GENESE", "Zone_HUD VL", "Zone_LONGIL", "Zone_MHK VL", "Zone_MILLWD", "Zone_N.Y.C.", "Zone_NORTH", "Zone_WEST"]
model=joblib.load("linear_predictor")


In [4]:
model=joblib.load("linear_predictor")

def predict_total_demand(month, hour): 
    ALL_ZONES = ["Zone_CAPITL", "Zone_CENTRL", "Zone_DUNWOD", "Zone_GENESE", "Zone_HUD VL", "Zone_LONGIL", "Zone_MHK VL", "Zone_MILLWD", "Zone_N.Y.C.", "Zone_NORTH", "Zone_WEST"]
    TOTAL_DEMAND = 0
    Demand_per_zone = {}
    for zone in ALL_ZONES:
        new_data = pd.DataFrame({"Month": Month, "Hour": Hour, "Zone_CAPITL": [zone=="Zone_CAPITL"], "Zone_CENTRL": [zone=="Zone_CENTRL"], "Zone_DUNWOD": [zone=="Zone_DUNWOD"], "Zone_GENESE": [zone=="Zone_GENESE"], "Zone_HUD VL": [zone=="Zone_HUD VL"], "Zone_LONGIL": [zone=="Zone_LONGIL"], "Zone_MHK VL": [zone=="Zone_MHK VL"], "Zone_MILLWD": [zone=="Zone_MILLWD"], "Zone_N.Y.C.": [zone=="Zone_N.Y.C."], "Zone_NORTH": [zone=="Zone_NORTH"], "Zone_WEST": [zone=="Zone_WEST"]})
        TOTAL_DEMAND += model.predict(new_data)[0]
        Demand_per_zone[zone] = model.predict(new_data)[0] 
    # print("Total Demand:", TOTAL_DEMAND)
    # print("Demand per zone:", Demand_per_zone)
    return TOTAL_DEMAND, Demand_per_zone
    

In [5]:
# Set up Gurobi environment
env = gp.Env(empty=True)
env.setParam('OutputFlag', 0)
env.start()

<gurobipy.Env, Parameter changes: OutputFlag=0>

In [6]:
def createOptimizationModel(month, hour, renewable=True):
    filtered_df = df[(df["Month"] == month) & (df["Hour"] == hour)].reset_index(drop=True) # filter by month and hour
    NUM_GENERATORS = len(filtered_df)
    TOTAL_DEMAND, Demand_per_zone = predict_total_demand(month, hour)
    # Initialize the model
    m = gp.Model(env=env)
    ### Decision Variables ###
    X = m.addVars((i for i in range(0, NUM_GENERATORS)), vtype=gp.GRB.BINARY, name='X')

    ### Constraints ###
    # Total Supply >= Demand Contraint
    m.addConstr(sum(X[i] * filtered_df["max_cap"][i] for i in range(0, NUM_GENERATORS)) >= TOTAL_DEMAND, name = "Total Supply >= Demand")
    # Renewable Supply >= 70% of Demand Constraint
    if renewable: 
        m.addConstr(sum(X[i] * filtered_df["renewable"][i] * filtered_df["max_cap"][i] for i in range(0, NUM_GENERATORS)) >= 0.7 * TOTAL_DEMAND, name = "Renewable Energy Supply")
    # Regional Energy Production Contraint 
    # for zone in ALL_ZONES:
    #    m.addConstr(sum(X[i] * df[zone][i] * df["max_cap"][i] for i in range(0, NUM_GENERATORS)) >= 0.5 * Demand_per_zone[zone], name = f"Regional Energy Supply {zone}")

    ### Objective Function ###
    # Minimize Cost
    m.setObjective(sum(filtered_df["Average LBMP"][i] * filtered_df["max_cap"][i] * X[i] for i in range(0, NUM_GENERATORS)),GRB.MINIMIZE)

    m.update() # Update model parameters
    m.write("OptimalSupply.lp") # Write model to file
    m.optimize() # Solve

    # print("\nObjective value: ", "%.2f" % m.getAttr("ObjVal"))
    return m.getAttr("ObjVal"), m.getAttr("X", X)


In [10]:
# Comparison of Renewable vs Non-Renewable Energy in 1 month 
month = 1
for hour in range(0, 24):
    obj_val1, x1 = createOptimizationModel(month, hour, renewable=False)
    obj_val2, x2 = createOptimizationModel(month, hour, renewable=True)
    print(obj_val1, obj_val2)   


465729.7741935484 478878.0
402616.41935483867 412289.51612903224
381228.9677419356 391356.0
367161.83870967745 372828.2580645161
338069.03225806454 342792.77419354836
330026.6129032258 336405.6129032258
358741.9354838709 365699.35483870964
451145.0322580645 471071.0322580644
443836.3548387097 453557.41935483867
749235.4516129033 770363.4516129033
511230.6129032259 519815.3870967743
508590.0322580645 520590.3548387097
498488.129032258 511463.22580645164
497285.41935483867 521000.6129032259
423512.41935483884 431072.32258064527
442006.6451612903 452873.4193548386
432988.77419354854 441796.32258064527
502977.0967741936 523037.6129032259
530677.6451612905 552973.1612903228
557566.8387096773 576659.6129032258
510525.1612903226 531402.1290322581
479033.0322580645 501915.48387096776
480314.5806451612 492162.0
456154.16129032255 460322.41935483867


In [15]:
# Comparison of Average Renewable vs Non-Renewable Energy in 1 year
for month in range(1, 13):
    total = 0
    for hour in range(0, 24):
        obj_val1, x1 = createOptimizationModel(month, hour, renewable=False)
        obj_val2, x2 = createOptimizationModel(month, hour, renewable=True)
        total += obj_val2
    print(total/24)   


476346.89516129036
392371.8690476191
267227.93548387097
265264.2847222222
233572.5067204301
383309.35138888884
537993.6639784947
257247.09274193548
480676.1430555556
370708.40860215056
401660.44305555546
1269721.7190860214
